In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/")

W0725 06:32:41.555330 139943368959872 deprecation.py:323] From <ipython-input-1-1d8ffff3e42a>:3: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0725 06:32:41.556877 139943368959872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0725 06:32:41.557821 139943368959872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instruction

Extracting MNIST_data/train-images-idx3-ubyte.gz


W0725 06:32:41.797849 139943368959872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0725 06:32:41.841245 139943368959872 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [2]:
import numpy as np

np.unique(mnist.train._labels)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

In [3]:
class CNN:
  def __init__(self, learning_rate):
    self.X = tf.placeholder(tf.float32, [None, 28, 28, 1])
    self.Y = tf.placeholder(tf.int32, [None])
    conv1 = tf.layers.conv2d(self.X, 
                             filters = 32,
                             kernel_size = 5,
                             strides=(1, 1),
                             padding='same',
                             activation = tf.nn.relu)
    
    pooling1 = tf.layers.max_pooling2d(conv1, pool_size = 2,
                                       strides = (2, 2),
                                       padding='valid')
    conv2 = tf.layers.conv2d(pooling1, 
                             filters = 64,
                             kernel_size = 5,
                             strides=(1, 1),
                             padding='same',
                             activation = tf.nn.relu)
    pooling2 = tf.layers.max_pooling2d(conv2, pool_size = 2,
                                       strides = (2, 2),
                                       padding='valid')
    
    conv3 = tf.layers.conv2d(pooling2, 
                             filters = 64,
                             kernel_size = 5,
                             strides=(1, 1),
                             padding='same',
                             activation = tf.nn.relu)
    
    pooling3 = tf.layers.max_pooling2d(conv3, pool_size = 2,
                                       strides = (2, 2),
                                       padding='valid')
    
    pooled = tf.layers.flatten(pooling3)
    self.logits = tf.layers.dense(pooled, 10)
    self.cost = tf.nn.sparse_softmax_cross_entropy_with_logits(
    labels = self.Y, logits = self.logits)
    self.cost = tf.reduce_mean(self.cost)
    self.optimizer = tf.train.AdamOptimizer(learning_rate) \
    .minimize(self.cost)
    correct_pred = tf.equal(
            tf.argmax(self.logits, 1, output_type = tf.int32), self.Y
        )
    self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
    
learning_rate = 1e-4
batch_size = 128
epoch = 10

tf.reset_default_graph()
sess = tf.InteractiveSession()
model = CNN(learning_rate)
sess.run(tf.global_variables_initializer())

from tqdm import tqdm

for e in range(epoch):
  pbar = tqdm(
        range(0, len(mnist.train._labels), batch_size), desc = 'train loop'
    )
  train_loss, train_acc, test_loss, test_acc = [], [], [], []
  for i in pbar:
    index = min(i + batch_size, len(mnist.train._labels))
    batch_x = mnist.train._images[i: index]
    batch_x = np.reshape(batch_x, (-1, 28, 28, 1))
    batch_y = mnist.train._labels[i: index]
    _, cost, accuracy = sess.run([model.optimizer, model.cost, model.accuracy],
                      feed_dict = {model.X: batch_x,
                                  model.Y: batch_y})
    train_loss.append(cost)
    train_acc.append(accuracy)
    pbar.set_postfix(cost = cost, accuracy = accuracy)

  pbar = tqdm(
        range(0, len(mnist.test._labels), batch_size), desc = 'test loop'
    )  
  for i in pbar:
    index = min(i + batch_size, len(mnist.test._labels))
    batch_x = mnist.test._images[i: index]
    batch_x = np.reshape(batch_x, (-1, 28, 28, 1))
    batch_y = mnist.test._labels[i: index]
    cost, accuracy = sess.run([model.cost, model.accuracy],
                      feed_dict = {model.X: batch_x,
                                  model.Y: batch_y})
    test_loss.append(cost)
    test_acc.append(accuracy)
    pbar.set_postfix(cost = cost, accuracy = accuracy)
  
  
  print('\nepoch: %d, avg train loss: %f, avg train accuracy: %f'%(e + 1,
                                                             np.mean(train_loss),
                                                             np.mean(train_acc)))
  print('epoch: %d, avg test loss: %f, avg test accuracy: %f'%(e + 1,
                                                             np.mean(test_loss),
                                                             np.mean(test_acc)))

W0725 07:00:01.374921 139943368959872 deprecation.py:323] From <ipython-input-3-00f2d0ab0ba1>:10: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0725 07:00:01.381520 139943368959872 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0725 07:00:01.612822 139943368959872 deprecation.py:323] From <ipython-input-3-00f2d0ab0ba1>:14: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0725 07:00:01.781600 139943368959872 deprecation.py:323] F


epoch: 1, avg train loss: 0.667287, avg train accuracy: 0.840410
epoch: 1, avg test loss: 0.201706, avg test accuracy: 0.940071


train loop:   3%|▎         | 12/430 [00:00<00:03, 119.90it/s, accuracy=0.953, cost=0.131]


epoch: 2, avg train loss: 0.178951, avg train accuracy: 0.947230
epoch: 2, avg test loss: 0.128190, avg test accuracy: 0.960245


train loop:   3%|▎         | 11/430 [00:00<00:03, 105.63it/s, accuracy=0.961, cost=0.112]

...


epoch: 8, avg train loss: 0.054488, avg train accuracy: 0.983386
epoch: 8, avg test loss: 0.054286, avg test accuracy: 0.981903


train loop:   3%|▎         | 11/430 [00:00<00:03, 109.70it/s, accuracy=0.984, cost=0.044] 


epoch: 9, avg train loss: 0.049025, avg train accuracy: 0.984912
epoch: 9, avg test loss: 0.050973, avg test accuracy: 0.982595


test loop: 100%|██████████| 79/79 [00:00<00:00, 196.07it/s, accuracy=1, cost=0.000163]



epoch: 10, avg train loss: 0.044467, avg train accuracy: 0.986511
epoch: 10, avg test loss: 0.048420, avg test accuracy: 0.982595


## Can try refer to tensorflow finished code
https://github.com/tensorflow/models/tree/master/research/slim # image recognition zoo
    
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md # object detection zoo